# Ensemble Learning

## Initial Imports

In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
!pip3 install sklearn

In [15]:
!pip3 install imblearn

In [16]:
pip install -U imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [17]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [18]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder, StandardScaler

## Read the CSV and Perform Basic Data Cleaning

In [31]:
# Load the data
#df = pd.read_csv("LoanStats_2019Q1.csv")
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [32]:
# Create our features
y = df.loc[:,['loan_status']].copy()
df.drop(columns=["issue_d", "loan_status", 'home_ownership',"hardship_flag", "debt_settlement_flag", "pymnt_plan", "verification_status"],inplace=True)
#df.head()
X = df.drop(["tax_liens"], axis=1)
# Create our target


In [39]:
X = X.select_dtypes(include='float64')


In [40]:
X

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,0.0,0.0,3.0,85.7,100.0,0.0,65687.0,38199.0,2000.0,61987.0
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,0.0,0.0,6.0,91.2,50.0,1.0,271427.0,60641.0,41200.0,49197.0
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,0.0,0.0,4.0,66.7,50.0,0.0,60644.0,45684.0,7500.0,43144.0
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,0.0,0.0,1.0,100.0,50.0,1.0,99506.0,68784.0,19700.0,76506.0
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,0.0,0.0,1.0,100.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,26000.0,9.60,0.0,0.0,9.0,0.0,2684.0,...,0.0,0.0,3.0,80.0,0.0,0.0,20625.0,6798.0,11300.0,5425.0
68813,12000.0,0.2727,368.37,63000.0,29.07,0.0,0.0,8.0,0.0,13314.0,...,0.0,0.0,5.0,96.2,0.0,0.0,87939.0,60350.0,13500.0,62939.0
68814,5000.0,0.1992,185.62,52000.0,14.86,0.0,0.0,5.0,1.0,3715.0,...,0.0,0.0,3.0,100.0,0.0,1.0,30592.0,18611.0,3600.0,18492.0
68815,40000.0,0.0646,1225.24,520000.0,9.96,0.0,1.0,21.0,0.0,59529.0,...,0.0,0.0,3.0,98.2,12.5,0.0,1033574.0,95958.0,100800.0,78634.0


In [41]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.0,0.052138,2.219423,95.057627,30.626217,0.125972,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.0,0.390633,1.897432,8.326426,33.631463,0.336732,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,20.000000,0.000000,0.000000,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.0,0.000000,1.000000,93.000000,0.000000,0.000000,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.0,0.000000,2.000000,100.000000,20.000000,0.000000,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.0,0.000000,3.000000,100.000000,50.000000,0.000000,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,0.0,18.000000,19.000000,100.000000,100.000000,4.000000,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [42]:
np.array(y)

array([['low_risk'],
       ['low_risk'],
       ['low_risk'],
       ...,
       ['low_risk'],
       ['low_risk'],
       ['low_risk']], dtype=object)

In [43]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   np.array(y), 
                                                   random_state=1, 
                                                   stratify=y)
X_train.shape

(51612, 75)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [44]:
# Create the StandardScaler instance
data_scaler = StandardScaler()

In [45]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
data_scaler.fit(X)


StandardScaler()

In [46]:
X

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,0.0,0.0,3.0,85.7,100.0,0.0,65687.0,38199.0,2000.0,61987.0
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,0.0,0.0,6.0,91.2,50.0,1.0,271427.0,60641.0,41200.0,49197.0
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,0.0,0.0,4.0,66.7,50.0,0.0,60644.0,45684.0,7500.0,43144.0
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,0.0,0.0,1.0,100.0,50.0,1.0,99506.0,68784.0,19700.0,76506.0
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,0.0,0.0,1.0,100.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,26000.0,9.60,0.0,0.0,9.0,0.0,2684.0,...,0.0,0.0,3.0,80.0,0.0,0.0,20625.0,6798.0,11300.0,5425.0
68813,12000.0,0.2727,368.37,63000.0,29.07,0.0,0.0,8.0,0.0,13314.0,...,0.0,0.0,5.0,96.2,0.0,0.0,87939.0,60350.0,13500.0,62939.0
68814,5000.0,0.1992,185.62,52000.0,14.86,0.0,0.0,5.0,1.0,3715.0,...,0.0,0.0,3.0,100.0,0.0,1.0,30592.0,18611.0,3600.0,18492.0
68815,40000.0,0.0646,1225.24,520000.0,9.96,0.0,1.0,21.0,0.0,59529.0,...,0.0,0.0,3.0,98.2,12.5,0.0,1033574.0,95958.0,100800.0,78634.0


In [53]:
data_scaler.fit(X[y])

StandardScaler()

In [55]:
# Scale the training and testing data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

In [62]:
X_test = scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [63]:
from imblearn.ensemble import BalancedRandomForestClassifier

In [64]:
# Resample the training data with the BalancedRandomForestClassifier
forest = BalancedRandomForestClassifier(n_estimators=500, random_state=78)
forest.fit(X_train, y_train)

BalancedRandomForestClassifier(n_estimators=500, random_state=78)

In [65]:
y_pred = forest.predict(X_test)

In [66]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.731494575179988

In [67]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   52,    35],
       [ 2306, 14812]], dtype=int64)

In [70]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.60      0.87      0.04      0.72      0.50        87
   low_risk       1.00      0.87      0.60      0.93      0.72      0.53     17118

avg / total       0.99      0.86      0.60      0.92      0.72      0.53     17205



In [72]:
# List the features sorted in descending order by feature importance
sorted(zip(forest.feature_importances_, X.columns), reverse=True)

[(0.07318965283564417, 'total_rec_prncp'),
 (0.06298534899917778, 'total_rec_int'),
 (0.06050223114126044, 'total_pymnt'),
 (0.0600346781860123, 'last_pymnt_amnt'),
 (0.05390631106472933, 'total_pymnt_inv'),
 (0.02935774412180493, 'int_rate'),
 (0.019368348410731435, 'dti'),
 (0.018992403683598245, 'annual_inc'),
 (0.018106566094753217, 'installment'),
 (0.018074780264873417, 'max_bal_bc'),
 (0.01741304482893629, 'mths_since_recent_inq'),
 (0.016697987430663194, 'revol_bal'),
 (0.016696399962091008, 'bc_open_to_buy'),
 (0.016687464390760378, 'mo_sin_old_il_acct'),
 (0.016359432113673158, 'out_prncp_inv'),
 (0.016334136736018006, 'mo_sin_old_rev_tl_op'),
 (0.016121469397473113, 'out_prncp'),
 (0.01606238641195404, 'bc_util'),
 (0.01588697279798545, 'avg_cur_bal'),
 (0.015429785024837156, 'tot_hi_cred_lim'),
 (0.015000105289727462, 'il_util'),
 (0.014955510017873562, 'all_util'),
 (0.014809246881800601, 'total_bal_il'),
 (0.014669746869607965, 'tot_cur_bal'),
 (0.014636760226400985, 'mth

### Easy Ensemble Classifier

In [74]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier

In [76]:
# Calculated the balanced accuracy score
forest = EasyEnsembleClassifier(n_estimators=100, random_state=78)
forest.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=500, random_state=78)

In [79]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   52,    35],
       [ 2306, 14812]], dtype=int64)

In [82]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.60      0.87      0.04      0.72      0.50        87
   low_risk       1.00      0.87      0.60      0.93      0.72      0.53     17118

avg / total       0.99      0.86      0.60      0.92      0.72      0.53     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    Balanced

2. Which model had the best recall score?

    Easy Ensabmle
3. Which model had the best geometric mean score?

    Balanced

4. What are the top three features?

    Balanced